<img src="assets/logos/Logo_main.png" width=500></img>
# Getting Started

The Purpose of this game day is to learn how to quickly develop and deploy machine learning models in a competitive environment. We will do this by developing models to play the game battleship!

## Game Description and Rules
Your team will create ML models to play "AI Command" which is like battleships! (as in "you sunk my battleship!") Each team will have a board they place their ships on and will try to guess the placement of the opposing team's ships. During each round of guess, the teams will pick a location on the opposing teams board which will be revealed as either a hit or miss. The first team to find all of the opposing teams ships wins!

In order to play the game, each team will need to create two ML models: a layout model that places ships on the board and a shoot model the guesses locations of the other teams ships. 

<img src="assets/build/battle-ship-game.png" width="600"></img>
### Rules
- __Board__: Each team in each game will place their ships on a rectangular game board and then guess/shoot the opposing ships board. Board size will be given to you. 
- __Layout__:
    - Ships: Ships can be __any shape__ as long as they follow the constraints given for the scenario. The details of these constraints will be in the Layout Endpoint Instructions
- __shoot__: Teams will take one shot at a time until one of the following happens:
    1. One team sinks all the ships of the opposing team
    2. The max number of shooting rounds is reached. The Max number of shooting rounds is equal to the area of the game board. So if a Game Board is 10x10 then there are max 100 shots per team per game
- __Scoring__: team scores will be tracked along three dimensions
    1. Games Won
    1. Total number of hits against opponents 
    1. Total number of hits by opponents

### Architecture

<img src="assets/build/architecture.png"></img>

The engine for game works like this:
1. An [Amazon CloudWatch](https://aws.amazon.com/cloudwatch/) alarm triggers the __launcher lambda__ every 30s (or some other time interval)
2. The Launcher Lambda reads from the __team [Amazon DynamoDB](https://aws.amazon.com/dynamodb/) table__ (which is written to when you register your team) and writes to the Game SQS queue where game config for each pair of teams is stored. 
3. The __Game lambda__ reads objects from the game queue and runs each game by
    1. calling each teams __layout endpoint__
    2. repeatedly calling each teams __shoot endpoint__ and updating score
    3. when the game is finished, it writes the game data to the __game [Amazon DynamoDB](https://aws.amazon.com/dynamodb/) table__
    4. if there are any errors from the teams, it writes the error to the team's __error SNS topic__
4. The __game DynamoDb__ table streams to the stream lambda which:
    1. writes the game data to an __Amazon Kinesis Firehose__ which writes the data to an __S3 bucket__ (for the teams to consume and download) and an __Elasticsearch database__ (for the game admins to debug).
        1. when new data is written to the S3 bucket a __SNS topic__ is trigger to notify teams of new data
    2. writes the score updates to a __redis cluster__

## Set Up
Your teams success will depend on:

- Ability to work together 
- Ability to deploy models quickly
- Use time effectively
- ML knowledge



###  Organize your Team
To help your team work together and deploy models quickly, we recommend that you organize your team into three groups: Operations, Defense, and Offense. Each group will have their own skills and responsibilities and will work with the other groups to win the game. 


1. __Defensive Group__: Responsible for researching and developing the Layout endpoint. Members who have strong data science skills or python programing skills will do well in this team.
1. __Offensive Group__: Responsible for researching and developing the Shooting endpoint. Members with strong machine learning skills will do will in this team.
1. __Operations__: Members who do not fit well into the Offensive of Defensive Groups can work in this group: 
    1. Managing/Deploying the Infrastructure 
    2. Deploying models
    3. Communicating errors and feedback to Layout and Shoot groups

You also need to give your team a name! 


### Schedule
To help your team use your time effectively, we suggest the following schedule for your team and each group. We also recommend you work in "small batch sizes". Try to do many small changes and deployments quickly instead of doing a few large deployments.

<img src="assets/build/schedule.png" width="800"></img>


## Code 

### Organization
The code required should already be installed onto your SageMaker notebook instance. The root directory of the project is in /home/ec2-user/SageMaker/GameDayRepo and any paths given in any following notebooks will be relative to this directory. In the root of the project there are a couple of directories and files to be aware of


```
bin/ #A directory of scripts to help you with common tasks
├── athena_info.sh
├── register_outputs.sh
├── startPipeline.js
├── testLayout.py
├── testShoot.py
├── upload_code.sh
├── download_data.sh
└── upload_data.sh

layout-config.json # Deploy/Train config for layout endpoint
shoot-config.json  # Deploy/Train config for shoot endpoint

containers/ #contains all the code for you Amazon [Amazon Sagemaker](https://aws.amazon.com/sagemaker/) training jobs and endpoints
├── Layout # code for the layout endpoint
│   ├── 2vs3
│   │   ├── host.py
│   │   ├── test.py
│   │   └── train.py
└── Shoot # code for the shoot endpoint
    ├── CNN # code for Deep learning CNN model
    │   ├── cnn.py
    │   ├── host.py
    │   ├── mock
    │   │   ├── data # directory to put sample data for local testing
    │   │   └── out
    │   ├── test.py
    │   ├── train.py
    ├── line # code for line shooting model
    │   ├── host.py
    │   ├── test.py
    │   └── train.py
    
notebooks/ # Jupyter notebooks/Tutorials for your team to help you through the day
├── Athena.ipynb
├── Endpoint_Reference.ipynb
├── defense
│   ├── Analytics.ipynb
│   └── getting_started.ipynb
├── getting_started.ipynb
├── offensive
│   ├── CNN.ipynb
│   └── getting_started.ipynb
└── operations
    └── getting_started.ipynb
    
data/ # Directory where the game data is downloaded and saved to.
```

### Deployment
Each group will work in an [Amazon Sagemaker](https://aws.amazon.com/sagemaker/) notebook instance. The following diagrams shows how each notebook push or pulls changes to the main codecommit repo (this repo was already created for you)
<img src="assets/build/overview.png" width="600"></img>


## Break into your groups!
break into your three groups and work through the corresponding tutorials

- [Operations Notebook](./operations/getting_started.ipynb)
- [Offensive Notebook](./offensive/getting_started.ipynb)
- [Defensive Notebook](./defense/getting_started.ipynb)
